In [1]:
##Python venv: search_agent_poc

In [1]:
##This module will take the serialized dictionary got out of PDF Parsing ann try to extract
##Semantic Knowldge like identifying 
## 1.Important Objects/Entities
## 2.Deduplicate Entities
## 3.Extracting Relations
## 4.Extract the main Ideas/Topics around Each Page
## 5.Link the different topics via diffrent entities/Objects
## 6.Break down the document by pages instead of Chunks .
## 7.If a page does not fit a chunk then chunk them extract information and then deduplicate the information across
## the page.

#Next Steps:
## 5.Try to Seggregate the BigPDF on Sections.
## 8.Try To Find Common Objects or ideas that link these sections.
## 9.Try 

In [16]:
import pickle
import pandas as pd
#rag_test_input_path='/home/matrix4284/MY_GEN_AI_PROJECTS/RAG/GraphRAG/graphrag-local-ollama/ragtest/input/'+file_name
import os
# importing shutil module
import shutil

from langchain_community.chat_models import ChatOllama

from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

from sentence_transformers import SentenceTransformer
#embeddings = SentenceTransformer("all-MiniLM-L6-v2")

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

import json
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
import json
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import XMLOutputParser

In [17]:
model_name="llama3.1"
book_text=''
page_text=''
file_name='Kubernetes_in_action_text_only'
extension='.txt'
start_page_idx=0
end_page_index=6
#full_filename=file_name+'_'+str(page_idx)+extension
#full_filename
pdf_enrichment_output_dir='./pdf_enriched_output/'

In [18]:
#LLM Model for Prompt Tuning
llm = ChatOllama(base_url="http://192.168.50.100:11434",model=model_name,temperature=0.15)

#embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
model_kwargs = {'device': 'cpu'}

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2",model_kwargs = model_kwargs)

##Define Vectorstore
vectorstore = Chroma(embedding_function=embeddings, persist_directory="./chroma_kubernetes_in_action")

In [19]:
# Load data (deserialized data)
#with open(pdf_enrichment_output_dir+'pdf_enriched_content_dict_stage2_phase2_final_fixed.pickle', 'rb') as handle:
#    document_dict_deserialized = pickle.load(handle)
    
with open(pdf_enrichment_output_dir+'pdf_enriched_content_dict_phase2_relation_extraction_478_final.pickle', 'rb') as handle:
    document_dict_deserialized = pickle.load(handle)    
    


In [20]:
document_dict_deserialized_stage3=document_dict_deserialized

In [21]:
type(document_dict_deserialized_stage3[4]['relationships'])

list

In [8]:
#document_dict_deserialized_stage3[460]

In [46]:
##Definitions of Individual Enrichment Modules######

def compose_summary_per_enity_relationships(relationships):
    ####To Be started By Kaustav Mukherjee 23/08/2024########
    
    parser = XMLOutputParser()

# Prompt


    
    print('Relationships From Input:')
    print(relationships)
               
    ###Convert Relationships to string to load into json#######
    #relationships_str=str(relationships)
    
    #print('Relationships String From Input:')
    #print(relationships_str.replace("'",'""')
    
    #relationships=relationships.split('[')[1]
    #relationships='['+relationships
    
    #relationships=relationships.split(']')[0]
    #relationships=relationships+']'
    
    #print('Relationships after cleaing:')
    #print(relationships)
    
    print('Relationships:')
    print(str(relationships).replace("'\"",'"').replace'\"\'','"'))
    
    
    relationships_json=json.loads()
    
    
                                  
    relationship_entity_sumary_map=[]
    

    #prompt = PromptTemplate(
    #        template="""You are a computer technology expert who has mastery in Kubernetes,Docker \n
    #        Machine Larning,Generative AI,Natural Language Understanding and Computer Vision.\n
    #        You have to deeply study a set of highlights and a list of entities for a page using the below rules.\n
    #        1.For every highlight study it very deeply and extract one or more entities from each highlight\n
    #        that matches entities provided as a part of input.\n
    #        2.Once this is done represent each highlight along with the list of extracted entities in a json \n
    #        3.Collate all the individual json into a list of json and output the list.\n
    #        There is no need to mention any header statement before the list .\n
    #        Output the list of json and nothing else no headers no footers.
    #        Here are the highlights: \n\n {context} \n\n and the entities: \n\n {entities} \n\n""",
    #        input_variables=["context","entities"],
    #        ###Introduced by Kaustav while experimenting with XMLParsers
    #        #partial_variables={"format_instructions": parser.get_format_instructions()},
    #)
    
   
        
    for relation in relationships_json:
        
        
        
        
        prompt = PromptTemplate(
            template="""You are a computer technology expert who has mastery in Kubernetes,Docker \n
            Machine Larning,Generative AI,Natural Language Understanding and Computer Vision.\n
            You have to deeply study all the relationships for a page using the below rules.\n
            1.For every relation item try to get the source entity along with its description, \n 
            destination entity along with description and relation description\n
            2.From the information of source entity ,destination entity  and \n
            description MUST come up with a sumary within 25 TO 50 words. \n
            3.Once this is done represent each summary in a json with tags named \n
            source,source_description,destination,destination_description,relation_description,summary_er.\n
            Output the json and nothing else no headers no footers.\n
            Here are the souce: \n\n {source}, the relation description: \n\n {relation} \n
            and the destination pod \n\n {destination} \n\n""",
            input_variables=["source","relation","destination"],)
    
    # Chain
    #Commented out By Kaustav 20/08/2024 as str output parser is giving some unwanted text #####
        chain = prompt | llm | StrOutputParser()
        
        print("source")
        print(relation['source_entity'])
        print("destination")
        print(relation["destination_entity"])
        print("relation")
        print(relation["destination_entity"])
        
        output = chain.invoke(
        {
                "source":relation['source_entity'],
                "relation": relation['description'],
                "destination": relation["destination_entity"],
        }
        )
        
        #print('From LLM OUTPUT:')
        #print(output)


    #Trying out XML Output parser by Kautsva added 20/08/2024####
    #chain = prompt | llm | parser
    
    ######Print highlights ###########
    #print('HIghlights:')
    #print(highlights)

    # Score
    #filtered_docs = []

    
    
        json_output = output
        #print('json_output from llm')
        #print(json_output)
        
        if '[' in json_output:
            json_output=json_output.split('[')[1]
            json_output='['+json_output
        elif '{' in json_output:
            json_output=json_output.split('{')[1]
            json_output='{'+json_output
            json_output='['+json_output
            
        #page_output_json=json.loads(output)
        
    
        #print('JSON OUTPUT:')
    #print(json_output)
        #except:
        #    json_output=json_output
        #    cotinue
        
        #try:
        if ']' in json_output:
            json_output=reverse(json_output)
            #print('Reversed JSON OUTPUT:')
            #print(json_output)
            json_output=json_output.split(']')[1]
            json_output=reverse(json_output)
            #json_output=json_output.rsplit(']')[-1]
            #page_output_json=json.loads(output)
            #print('JSON OUTPUT:')
            #print(json_output)
            json_output= json_output + ']'
            #print('JSON OUTPUT:')
            #print(json_output) 
        elif '}' in json_output:
            json_output=json_output.split('}')[0]
            json_output=json_output+'}'
            json_output=json_output+']'
        
        
        #print('cleaned json_output from llm')
        #print(json_output)
        
        #except:
        #    json_output=json_output
        #    cotinue
    
        #print("JSON OUTPUT:")
        #print(json_output)
        #page_output_json=json.loads(output)
        #return page_output_json
        relationship_entity_sumary_map.append(json_output) 
            
    return relationship_entity_sumary_map 
    
        #return page_output_json
        #return output

    
# Function to reverse a string
def reverse(string):
    string = string[::-1]
    return string
    
##Definitions of Individual Enrichment Modules######



def entity_collector_per_page(entity_lst):

    entities=[]
    
    for entity in entity_lst:
        print("Entity:")
        print(entity)
        entity_name=entity['entity']
        entities.append(entity_name)
    return list(set(entities))


def enrich_page(page_idx):
    
    #print("Page Number")
    #print(page_idx)
    
    
    
    
    ########################################Summary Extraction Enrichment###########################################
    #page_text=document_dict_deserialized_stage3[page_idx]['text']
    ##Summary Extraction Enrichment
    #page_summary_txt=extract_summary_per_page(page_text)
    
    #print("Page Summary Text")
    #print(page_summary_txt)
    
    ######Add Summary as a part of pdf structured dictionary list in order summarization enrichment to data######### 
    #document_dict_deserialized_stage2[page_idx]['summary']=json.loads(page_summary_txt.strip())["summary"]
    
    ################################################################################################################
    
    ######################################Highlight Extraction######################################################
    
    ##Summary Extraction Enrichment
    #page_highlights_json_lst=extract_highlights_per_page('""'+page_text+'""')
    
    #print("Page Highlights")
    #print(page_highlights_json_lst)
    
    #page_highlights_json_lst=json.loads(page_highlights_json_lst)
    
    ######Add Summary as a part of pdf structured dictionary list in order summarization enrichment to data######### 
    #document_dict_deserialized_stage3[page_idx]['highlights']=page_highlights_json_lst
    
    ################################################################################################################
    
    
    #################################Map Highlights with individual entities for each page#########################
    relationships_=[]
    
    if document_dict_deserialized_stage3[page_idx]['relationships']:
        relationships=document_dict_deserialized_stage3[page_idx]['relationships']
    
    
    relationships=document_dict_deserialized_stage3[page_idx]['relationships']
    
     
 
    
    #for relationship in relationships:
    #    relationships_summary_lst.append(hightlight['highlight'])
    
    
    ##Highlight to highlight mapping Enrichment
    entity_relation_summ_text=compose_summary_per_enity_relationships(relationships)
    
    if entity_relation_summ_text is not None:
        
        
        #entity_relation_summ_json=json.loads(entity_relation_summ_text)
        #print("Summarization of Entity,Relation,Tuple")
        #print(entity_relation_summ_text)
        #print(type(entity_relation_summ_text))
        #print(entity_relation_summ_json)
    
    ######Add Summary as a part of pdf structured dictionary list in order summarization enrichment to data######### 
        document_dict_deserialized_stage3[page_idx]['summary_rel']=entity_relation_summ_text
    

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 91)

In [47]:
for idx in range(start_page_idx,end_page_index):
   
        enrich_page(idx)
        print("Done for page number:"+str(idx))

Relationships From Input:
[{'source_entity': '"Network namespace"', 'description': 'manages', 'destination_entity': '"Linux namespaces"'}, {'source_entity': '"Network namespace"', 'description': 'allocates', 'destination_entity': '"Port space"'}, {'source_entity': '"UTS namespace"', 'description': 'manages', 'destination_entity': '"Linux namespaces"'}, {'source_entity': '"Kubernetes"', 'description': 'orchestrates', 'destination_entity': '"Pods"'}, {'source_entity': '"Docker"', 'description': 'runs', 'destination_entity': '"Containers"'}, {'source_entity': '"Kubernetes"', 'description': 'manages', 'destination_entity': '"Volume"'}, {'source_entity': '"PID namespace"', 'description': 'manages', 'destination_entity': '"Linux namespaces"'}, {'source_entity': '"IPC namespace"', 'description': 'manages', 'destination_entity': '"Linux namespaces"'}]
Relationships:
[{'source_entity': "Network namespace"', 'description': 'manages', 'destination_entity': "Linux namespaces"'}, {'source_entity': 

TypeError: loads() missing 1 required positional argument: 's'

In [35]:
document_dict_deserialized_stage3[164]

{'page': 253,
 'img_cnt': 0,
 'img_npy_lst': [],
 'text': '221\nUsing Secrets to pass sensitive data to containers\nTESTING WHETHER NGINX IS USING THE CERT AND KEY FROM THE SECRET\nOnce the pod is running, you can see if it’s serving HTTPS traffic by opening a port-\nforward tunnel to the pod’s port 443 and using it to send a request to the server\nwith curl: \n$ kubectl port-forward fortune-https 8443:443 &\nForwarding from 127.0.0.1:8443 -> 443\nForwarding from [::1]:8443 -> 443\n$ curl https://localhost:8443 -k\nIf you configured the server properly, you should get a response. You can check the\nserver’s certificate to see if it matches the one you generated earlier. This can also be\ndone with curl by turning on verbose logging using the -v option, as shown in the fol-\nlowing listing.\n$ curl https://localhost:8443 -k -v\n* About to connect() to localhost port 8443 (#0)\n*   Trying ::1...\n* Connected to localhost (::1) port 8443 (#0)\n* Initializing NSS with certpath: sql:/etc/pk

In [21]:
import pickle

# Store data (serialize in a pickle) upto page 102
with open('./pdf_enriched_output/pdf_enriched_content_dict_stage2_phase3_final_163.pickle', 'wb') as handle:
    pickle.dump(document_dict_deserialized_stage3, handle, protocol=pickle.HIGHEST_PROTOCOL)